In [1]:
# import files

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
from datetime import date, timedelta
import pandas as pd

import time

In [2]:
# Global variables
search_field_term_job = "data scientist"
search_field_term_location = "New York, NY"

In [3]:
# define driver
driver = webdriver.Chrome()

# open url
driver.get("https://www.simplyhired.com/")

# wait for the page to load
driver.implicitly_wait(5)


# find and click on search field location
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R5bakt9fbqm:"]')
search_field.click()

# clear search field
#search_field.clear()
search_field.send_keys(Keys.CONTROL + "a")
search_field.send_keys(Keys.DELETE)

# enter search term job
search_field.send_keys(search_field_term_location)




# find and click on search field job
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R3bakt9fbqm:"]')
search_field.click()

# wait for the page to load
driver.implicitly_wait(5)

# enter search term job
search_field.send_keys(search_field_term_job)

element = driver.switch_to.active_element
element.send_keys(Keys.ESCAPE)

# wait for the page to load
driver.implicitly_wait(5)



# submit form
# search_field.submit()
search_field.send_keys(Keys.ENTER)

Job Title, Company, Location, link, Job_description, qualification as string

In [7]:
def pullJobData():
    webElementJobList = driver.find_element(By.XPATH, '//*[@id="job-list"]')
    num_rows = len(webElementJobList.find_elements(By.XPATH, '//li[@class="css-0"]'))
    wait = WebDriverWait(driver, 10)
    

    job_list = []

    for i in range(1, num_rows+1):
        driver.implicitly_wait(5)
        time.sleep(5)

        for tries in range(2):

            try:
                jobTitle = webElementJobList.find_element(By.XPATH, f'//li[@class="css-0"][{i}]//a').text
                break
            except Exception as e:
                print('job title not found', e)


        for tries in range(2):
            try:
                link = webElementJobList.find_element(By.XPATH, f'//li[@class="css-0"][{i}]//a')\
                    .get_attribute('href')
                break
            except:
                print('link not found')



        # click on job
        time.sleep(2)

        for tries in range(2):
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, f'//li[@class="css-0"][{i}]//a')))
                webElementJobList.find_element(By.XPATH, f'//li[@class="css-0"][{i}]//a').click()
                break
            except:
                print('link not clickable')
        
        time.sleep(3)

        # store values  

        # description
        for tries in range(2):
            xpath = '//*[@data-testid="viewJobBodyJobFullDescriptionContent"]'
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
                describtion = driver.find_element(By.XPATH, xpath).text
                break
            except:
                print('description not found')

        

        # list in order of appearance company, location, hours, salary, date posted
        # should be 2 or more
        job_desc = {'company':'',
                    'location':'',
                    'salary':'',
                    'date posted':''
                    }
        
        for tries in range(2):
            xpath = '//*[@data-testid="viewJobHeadingContainer"]//span[@data-testid="detailText"]'
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
                job_desc['company'], job_desc['location']  = [ele.text for ele in driver.\
                       find_elements(By.XPATH, xpath)]
                break
            except:
                print('company and location not found')

            
        for tries in range(2):
            
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
                job_desc['date posted'] = driver.find_element(By.XPATH, '//*[@data-testid="viewJobBodyJobPostingTimestamp"]/span').text

                if job_desc['date posted'] != '':
                    day = re.findall(r'\d+', job_desc['date posted'])
                    day = int(day[0])

                    day = date.today() - timedelta(days = day)
                    job_desc['date posted'] = day
                break
            except:
                print('date posted not found')

        for tries in range(2):

            try:

                job_desc['salary']  = driver.find_element(By.XPATH, '//*[@data-testid="viewJobBodyJobCompensation"]').text
                break
            except:
                print('company and location not found')

            
        

        # qualification
        xpath = '//*[@id="__next"]/div/main/div/div[2]/div/div/div[2]/div/aside/div/div[1]/div/div[3]/div/div/ul//li'
        for tries in range(2):
            try:
                qualifications = [detail.text for detail in driver.find_elements(By.XPATH, xpath)]
                qualifications = "||".join(qualifications)
                break
            except:
                print('qualifications not found')




        job_list.append([jobTitle, job_desc['company'],job_desc['location'],job_desc['salary'],\
                    job_desc['date posted'], link, describtion, qualifications ])
    
    return job_list

In [8]:
jobs = []

for _ in range(10):
    jobs.extend(pullJobData())

    try:
        next_page = driver.find_element(By.XPATH, '//*[@id="__next"]//main//nav//a[last()]')
        next_page.click()
    except:
        print('No more pages')

    # let page load(driver.implicitly_wait does not work
    time.sleep(5)




date posted not found
date posted not found
date posted not found
date posted not found
company and location not found
company and location not found
date posted not found
date posted not found
company and location not found
company and location not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
date posted not found
company and location not found
company and location not found
company and location not found
company and location not found
company and location not found
company and location not found
company and location not found
company and location not found
date posted 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="job-list"]"}
  (Session info: chrome=120.0.6099.216); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65E3D2142+3514994]
	(No symbol) [0x00007FF65DFF0CE2]
	(No symbol) [0x00007FF65DE976AA]
	(No symbol) [0x00007FF65DEE1860]
	(No symbol) [0x00007FF65DEE197C]
	(No symbol) [0x00007FF65DF24EE7]
	(No symbol) [0x00007FF65DF0602F]
	(No symbol) [0x00007FF65DF228F6]
	(No symbol) [0x00007FF65DF05D93]
	(No symbol) [0x00007FF65DED4BDC]
	(No symbol) [0x00007FF65DED5C64]
	GetHandleVerifier [0x00007FF65E3FE16B+3695259]
	GetHandleVerifier [0x00007FF65E456737+4057191]
	GetHandleVerifier [0x00007FF65E44E4E3+4023827]
	GetHandleVerifier [0x00007FF65E1204F9+689705]
	(No symbol) [0x00007FF65DFFC048]
	(No symbol) [0x00007FF65DFF8044]
	(No symbol) [0x00007FF65DF

In [9]:
jobs = pd.DataFrame(jobs, columns = ['jobTitle', 'company', 'location', 'salary', 'date posted', 'link', 'describtion', 'qualifications'])
jobs.head()

,jobTitle,company,location,salary,date posted,link,describtion,qualifications
0,Data Scientist – NLP,Solytics Partners LLC,"New York, NY","$90,000 - $120,000 a year",,https://www.simplyhired.com/job/qCZn6oDLj-JMtw...,Solytics Partners provide products and service...,Statistics||MATLAB||Computer Science||Cassandr...
1,Senior Data Analyst / Manager,Maimonides Medical Center,"Brooklyn, NY","$107,000 - $117,000 a year",2024-01-08,https://www.simplyhired.com/job/RO64y1Xr-cTre2...,Senior Data Analyst/Manager\nTHE BEST CAREERS....,Microsoft Excel||Computer Science||5 years||Ta...
2,"Sr Distinguished Engineer, Generative AI Syste...",Capital One,"New York, NY",,,https://www.simplyhired.com/job/2fp1RQhYCLZy4w...,"Locations: VA - McLean, United States of Ameri...",TensorFlow||Computer science||Cloud infrastruc...
3,Deep Machine Learning Engineer (Expert Level P...,Career Developers,"New York, NY","$300,000 - $600,000 a year",,https://www.simplyhired.com/job/YAMJY8OAGOtA6B...,Refer a friend: Referral fee program\nSmall Mo...,7 years||Machine learning||Deep learning||Python
4,Data Science Manager,Plymouth Rock Assurance,"Woodbridge, NJ",Estimated: $138K - $175K a year,2024-01-03,https://www.simplyhired.com/job/vpqawg4-7wdJMI...,Overview:\n\nPlymouth Rock has been awarded th...,Statistics||Doctoral degree||SAS||5 years||R||...


In [10]:
jobs.shape

(120, 8)

In [13]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jobTitle        120 non-null    object
 1   company         120 non-null    object
 2   location        120 non-null    object
 3   salary          120 non-null    object
 4   date posted     120 non-null    object
 5   link            120 non-null    object
 6   describtion     120 non-null    object
 7   qualifications  120 non-null    object
dtypes: object(8)
memory usage: 7.6+ KB


In [12]:
jobs.to_csv('simplyHired_scrape.csv', index = False )

In [ ]:
driver.close()